# 第 5 章：LSTM 深度解析

> Long Short-Term Memory - 解决 RNN 长期依赖问题的里程碑

---

## 本章目标

完成本章后，你将能够：

- [ ] 理解 LSTM 的核心创新：细胞状态和门控机制
- [ ] 详细理解遗忘门、输入门、输出门的作用
- [ ] 用 NumPy 从零实现 LSTM
- [ ] 分析 LSTM 如何解决梯度消失问题
- [ ] 在长期依赖任务上验证 LSTM 的效果

---

## LSTM 的历史背景

- **1997年**: Hochreiter & Schmidhuber 发明 LSTM
- **目的**: 解决 RNN 的梯度消失问题
- **至今**: 仍是序列建模的重要架构（直到 Transformer 出现）

```
RNN 的问题:                    LSTM 的解决方案:
┌─────────────────┐           ┌─────────────────────────────┐
│ 梯度消失       │ ────────▶ │ 细胞状态 (Cell State)        │
│ 无法记住       │           │ 信息高速公路，梯度直接传递   │
│ 长期信息       │           ├─────────────────────────────┤
│                │ ────────▶ │ 门控机制 (Gates)             │
│ 信息无差别     │           │ 学习何时记住/遗忘/输出       │
│ 混合在一起     │           │                              │
└─────────────────┘           └─────────────────────────────┘
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)

print("环境准备完成！")

---

## 第一部分：LSTM 核心思想

### 1.1 RNN vs LSTM 结构对比

**标准 RNN**:
- 只有一个隐状态 $h_t$
- 所有信息混合在一起
- 梯度必须通过 tanh，容易消失

**LSTM**:
- 两个状态：隐状态 $h_t$ + 细胞状态 $c_t$
- 细胞状态是"记忆高速公路"
- 三个门控制信息流动

In [ ]:
# 可视化：RNN vs LSTM 结构

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# ========================================
# 左图：标准 RNN
# ========================================
ax = axes[0]
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)

# RNN 单元
rect = plt.Rectangle((3, 3), 4, 4, fill=True, facecolor='lightblue', 
                       edgecolor='blue', linewidth=2)
ax.add_patch(rect)
ax.text(5, 5, 'tanh', ha='center', va='center', fontsize=16, fontweight='bold')

# 输入 x_t
ax.annotate('', xy=(3, 5), xytext=(1, 5),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))
ax.text(0.5, 5, '$x_t$', fontsize=14, va='center')

# 输出 h_t
ax.annotate('', xy=(9, 5), xytext=(7, 5),
            arrowprops=dict(arrowstyle='->', color='red', lw=2))
ax.text(9.2, 5, '$h_t$', fontsize=14, va='center')

# 循环连接 h_{t-1}
ax.annotate('', xy=(5, 3), xytext=(5, 1),
            arrowprops=dict(arrowstyle='->', color='orange', lw=2))
ax.text(5.5, 0.5, '$h_{t-1}$', fontsize=12)

ax.set_title('标准 RNN: 简单但有缺陷', fontsize=14, fontweight='bold')
ax.text(5, 8.5, '问题: 所有信息混在一起\n梯度通过 tanh 容易消失', 
        ha='center', fontsize=11, color='red',
        bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
ax.axis('off')

# ========================================
# 右图：LSTM
# ========================================
ax = axes[1]
ax.set_xlim(0, 14)
ax.set_ylim(0, 10)

# LSTM 单元（大方框）
rect = plt.Rectangle((2, 2), 10, 6, fill=True, facecolor='lightyellow', 
                       edgecolor='orange', linewidth=2)
ax.add_patch(rect)

# 细胞状态通道（顶部）
ax.annotate('', xy=(12, 7), xytext=(2, 7),
            arrowprops=dict(arrowstyle='->', color='purple', lw=3))
ax.text(7, 7.5, '$c_t$ (细胞状态)', ha='center', fontsize=11, color='purple')

# 三个门（小方框）
gate_colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
gate_names = ['遗忘门\n$f_t$', '输入门\n$i_t$', '输出门\n$o_t$']
gate_x = [3.5, 6, 8.5]

for x, color, name in zip(gate_x, gate_colors, gate_names):
    rect = plt.Rectangle((x-0.6, 3.5), 1.2, 1.5, fill=True, 
                          facecolor=color, edgecolor='black', linewidth=1.5, alpha=0.7)
    ax.add_patch(rect)
    ax.text(x, 4.25, name, ha='center', va='center', fontsize=9)

# 输入
ax.annotate('', xy=(2, 4), xytext=(0.5, 4),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))
ax.text(0, 4, '$x_t$', fontsize=12, va='center')

# 输出 h_t
ax.annotate('', xy=(13.5, 4), xytext=(12, 4),
            arrowprops=dict(arrowstyle='->', color='red', lw=2))
ax.text(13.7, 4, '$h_t$', fontsize=12, va='center')

# 循环输入
ax.annotate('', xy=(7, 2), xytext=(7, 0.5),
            arrowprops=dict(arrowstyle='->', color='orange', lw=2))
ax.text(7.5, 0.3, '$h_{t-1}, c_{t-1}$', fontsize=10)

ax.set_title('LSTM: 门控记忆单元', fontsize=14, fontweight='bold')
ax.text(7, 9, '创新: 细胞状态 + 三个门\n梯度可以直接通过细胞状态传递', 
        ha='center', fontsize=11, color='green',
        bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
ax.axis('off')

plt.tight_layout()
plt.show()

### 1.2 LSTM 的核心创新

#### 创新 1：细胞状态 (Cell State)

细胞状态 $c_t$ 是 LSTM 的"记忆高速公路"：

$$c_t = f_t \odot c_{t-1} + i_t \odot \tilde{c}_t$$

- $\odot$ 表示逐元素乘法
- $f_t$ 决定保留多少旧记忆
- $i_t \odot \tilde{c}_t$ 决定写入多少新信息

**关键洞察**：$c_t$ 的更新是**加法**而非**乘法**，梯度可以直接传递！

#### 创新 2：门控机制 (Gating Mechanism)

三个门都使用 sigmoid 激活，输出在 [0, 1] 之间：

| 门 | 公式 | 作用 |
|---|---|---|
| 遗忘门 $f_t$ | $\sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$ | 决定丢弃哪些旧信息 |
| 输入门 $i_t$ | $\sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$ | 决定更新哪些新信息 |
| 输出门 $o_t$ | $\sigma(W_o \cdot [h_{t-1}, x_t] + b_o)$ | 决定输出哪些信息 |

---

## 第二部分：LSTM 完整公式

### 2.1 LSTM 前向传播公式

输入：$x_t$（当前输入），$h_{t-1}$（上一时刻隐状态），$c_{t-1}$（上一时刻细胞状态）

**Step 1: 计算遗忘门**
$$f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$$

**Step 2: 计算输入门和候选值**
$$i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$$
$$\tilde{c}_t = \tanh(W_c \cdot [h_{t-1}, x_t] + b_c)$$

**Step 3: 更新细胞状态**
$$c_t = f_t \odot c_{t-1} + i_t \odot \tilde{c}_t$$

**Step 4: 计算输出门和隐状态**
$$o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)$$
$$h_t = o_t \odot \tanh(c_t)$$

输出：$h_t$（隐状态），$c_t$（细胞状态）

In [ ]:
# 可视化：LSTM 数据流详解

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

def draw_lstm_step(ax, title, highlight=None):
    """绘制 LSTM 单步示意图"""
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 8)
    
    # 细胞状态通道
    ax.annotate('', xy=(9, 6.5), xytext=(1, 6.5),
                arrowprops=dict(arrowstyle='->', color='purple', lw=3))
    ax.text(0.5, 6.5, '$c_{t-1}$', fontsize=11, va='center')
    ax.text(9.5, 6.5, '$c_t$', fontsize=11, va='center')
    
    # 隐状态输入
    ax.annotate('', xy=(2, 2), xytext=(0.5, 2),
                arrowprops=dict(arrowstyle='->', color='orange', lw=2))
    ax.text(0, 2, '$h_{t-1}$', fontsize=11, va='center')
    
    # 当前输入
    ax.annotate('', xy=(2, 1), xytext=(0.5, 1),
                arrowprops=dict(arrowstyle='->', color='green', lw=2))
    ax.text(0, 1, '$x_t$', fontsize=11, va='center')
    
    # 遗忘门
    color = 'red' if highlight == 'forget' else 'lightcoral'
    rect = plt.Rectangle((2.5, 5), 1.5, 1.2, fill=True, facecolor=color, 
                          edgecolor='darkred', linewidth=2)
    ax.add_patch(rect)
    ax.text(3.25, 5.6, '$f_t$', ha='center', va='center', fontsize=12, fontweight='bold')
    ax.text(3.25, 4.5, '遗忘门', ha='center', fontsize=9)
    
    # 输入门 + 候选值
    color = 'green' if highlight == 'input' else 'lightgreen'
    rect = plt.Rectangle((4.5, 5), 1.5, 1.2, fill=True, facecolor=color,
                          edgecolor='darkgreen', linewidth=2)
    ax.add_patch(rect)
    ax.text(5.25, 5.6, '$i_t$', ha='center', va='center', fontsize=12, fontweight='bold')
    
    rect = plt.Rectangle((4.5, 3.5), 1.5, 1.2, fill=True, facecolor='lightyellow',
                          edgecolor='olive', linewidth=2)
    ax.add_patch(rect)
    ax.text(5.25, 4.1, '$\\tilde{c}_t$', ha='center', va='center', fontsize=12)
    ax.text(5.25, 2.8, '输入门\n+候选', ha='center', fontsize=9)
    
    # 输出门
    color = 'blue' if highlight == 'output' else 'lightblue'
    rect = plt.Rectangle((7, 3.5), 1.5, 1.2, fill=True, facecolor=color,
                          edgecolor='darkblue', linewidth=2)
    ax.add_patch(rect)
    ax.text(7.75, 4.1, '$o_t$', ha='center', va='center', fontsize=12, fontweight='bold')
    ax.text(7.75, 2.8, '输出门', ha='center', fontsize=9)
    
    # 输出
    ax.annotate('', xy=(9.5, 4), xytext=(8.5, 4),
                arrowprops=dict(arrowstyle='->', color='red', lw=2))
    ax.text(9.7, 4, '$h_t$', fontsize=11, va='center')
    
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.axis('off')

# 四个步骤
draw_lstm_step(axes[0, 0], 'Step 1: 遗忘门决定丢弃什么', 'forget')
axes[0, 0].text(5, 0.5, '$f_t = \\sigma(W_f \\cdot [h_{t-1}, x_t] + b_f)$', 
                ha='center', fontsize=11, 
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

draw_lstm_step(axes[0, 1], 'Step 2: 输入门决定写入什么', 'input')
axes[0, 1].text(5, 0.5, '$i_t = \\sigma(...), \\tilde{c}_t = tanh(...)$', 
                ha='center', fontsize=11,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

draw_lstm_step(axes[1, 0], 'Step 3: 更新细胞状态')
axes[1, 0].text(5, 0.5, '$c_t = f_t \\odot c_{t-1} + i_t \\odot \\tilde{c}_t$', 
                ha='center', fontsize=11,
                bbox=dict(boxstyle='round', facecolor='lightpurple', alpha=0.8))

draw_lstm_step(axes[1, 1], 'Step 4: 输出门决定输出什么', 'output')
axes[1, 1].text(5, 0.5, '$h_t = o_t \\odot tanh(c_t)$', 
                ha='center', fontsize=11,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

### 2.2 门的直观理解

用一个例子来理解三个门的作用：

**场景**：阅读一段文本，预测下一个词

```
"The cat, which was very hungry and had been looking for food 
 all day long, finally sat on the ___"
```

| 门 | 作用 | 例子 |
|---|---|---|
| 遗忘门 $f_t$ | 忘记不相关信息 | 忘记 "which was very hungry..." 这些修饰语 |
| 输入门 $i_t$ | 记住新的重要信息 | 记住主语是 "cat"，动作是 "sat" |
| 输出门 $o_t$ | 输出当前需要的信息 | 输出与 "sat on the" 相关的上下文，预测 "mat" |

In [ ]:
def gate_behavior_demo():
    """
    演示门的行为：sigmoid 输出如何控制信息流
    """
    print("=" * 60)
    print("门控机制演示")
    print("=" * 60)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # 信息向量
    info = np.array([0.8, -0.5, 0.3, 0.9, -0.7, 0.2, 0.6, -0.4])
    
    # 三种门值
    gate_scenarios = [
        ('全部保留 (gate ≈ 1)', np.ones(8) * 0.95),
        ('全部丢弃 (gate ≈ 0)', np.ones(8) * 0.05),
        ('选择性保留', np.array([0.9, 0.1, 0.8, 0.95, 0.05, 0.3, 0.85, 0.15]))
    ]
    
    x = np.arange(len(info))
    width = 0.35
    
    for ax, (title, gate) in zip(axes, gate_scenarios):
        # 原始信息
        bars1 = ax.bar(x - width/2, info, width, label='原始信息', color='steelblue', alpha=0.7)
        
        # 门控后的信息
        gated_info = info * gate
        bars2 = ax.bar(x + width/2, gated_info, width, label='门控后', color='coral', alpha=0.7)
        
        # 在顶部显示门值
        for i, g in enumerate(gate):
            ax.text(i, max(info) + 0.15, f'{g:.2f}', ha='center', fontsize=8, color='green')
        
        ax.axhline(y=0, color='k', linewidth=0.5)
        ax.set_xlabel('信息维度', fontsize=10)
        ax.set_ylabel('值', fontsize=10)
        ax.set_title(title, fontsize=12, fontweight='bold')
        ax.legend(loc='lower right', fontsize=9)
        ax.set_xticks(x)
        ax.text(3.5, max(info) + 0.35, '门值 (绿色)', ha='center', fontsize=9, color='green')
    
    plt.tight_layout()
    plt.show()
    
    # 数值示例
    print("\n数值示例:")
    print("-" * 40)
    print(f"原始信息: {info}")
    print(f"\n选择性门值: {gate_scenarios[2][1]}")
    print(f"门控结果: {info * gate_scenarios[2][1]}")
    print("\n观察: 门值接近 1 的维度被保留，接近 0 的被丢弃")


gate_behavior_demo()

---

## 第三部分：NumPy 从零实现 LSTM

### 3.1 LSTM Cell 实现

In [ ]:
class LSTMCell:
    """
    LSTM 单元的 NumPy 实现
    
    公式:
        f_t = sigmoid(W_f @ [h_{t-1}, x_t] + b_f)    # 遗忘门
        i_t = sigmoid(W_i @ [h_{t-1}, x_t] + b_i)    # 输入门
        c_tilde = tanh(W_c @ [h_{t-1}, x_t] + b_c)   # 候选细胞状态
        c_t = f_t * c_{t-1} + i_t * c_tilde          # 更新细胞状态
        o_t = sigmoid(W_o @ [h_{t-1}, x_t] + b_o)    # 输出门
        h_t = o_t * tanh(c_t)                         # 更新隐状态
    
    参数:
        input_size: 输入特征维度
        hidden_size: 隐藏状态维度
    """
    
    def __init__(self, input_size, hidden_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # ========================================
        # 参数初始化
        # 将四个门的权重合并为一个大矩阵（提高效率）
        # ========================================
        
        # 合并后的权重矩阵
        # 包含: W_f, W_i, W_c, W_o
        # 输入: [h_{t-1}, x_t], 维度: hidden_size + input_size
        # 输出: 4 * hidden_size (四个门的输出)
        
        concat_size = hidden_size + input_size
        
        # Xavier 初始化
        scale = np.sqrt(2.0 / (concat_size + hidden_size))
        
        # 权重矩阵, shape: (4 * hidden_size, hidden_size + input_size)
        self.W = np.random.randn(4 * hidden_size, concat_size) * scale
        
        # 偏置, shape: (4 * hidden_size,)
        self.b = np.zeros(4 * hidden_size)
        
        # 注意: 遗忘门偏置初始化为 1（让网络一开始倾向于记住信息）
        self.b[0:hidden_size] = 1.0
    
    def sigmoid(self, x):
        """Sigmoid 激活函数: σ(x) = 1 / (1 + e^{-x})"""
        # 数值稳定版本
        return np.where(x >= 0, 
                        1 / (1 + np.exp(-x)), 
                        np.exp(x) / (1 + np.exp(x)))
    
    def forward(self, x_t, h_prev, c_prev):
        """
        LSTM 单步前向传播
        
        参数:
            x_t: 当前输入, shape: (input_size,)
            h_prev: 上一时刻隐状态, shape: (hidden_size,)
            c_prev: 上一时刻细胞状态, shape: (hidden_size,)
        
        返回:
            h_t: 当前隐状态, shape: (hidden_size,)
            c_t: 当前细胞状态, shape: (hidden_size,)
        """
        H = self.hidden_size
        
        # ========================================
        # Step 1: 拼接输入 [h_{t-1}, x_t]
        # ========================================
        concat = np.concatenate([h_prev, x_t])  # shape: (H + input_size,)
        
        # ========================================
        # Step 2: 一次性计算所有门的线性变换
        # ========================================
        gates = self.W @ concat + self.b  # shape: (4H,)
        
        # ========================================
        # Step 3: 拆分并应用激活函数
        # ========================================
        # 拆分成四个部分，每个 shape: (H,)
        f_gate = self.sigmoid(gates[0*H : 1*H])      # 遗忘门
        i_gate = self.sigmoid(gates[1*H : 2*H])      # 输入门
        c_tilde = np.tanh(gates[2*H : 3*H])          # 候选细胞状态
        o_gate = self.sigmoid(gates[3*H : 4*H])      # 输出门
        
        # ========================================
        # Step 4: 更新细胞状态
        # c_t = f_t * c_{t-1} + i_t * c_tilde
        # ========================================
        # 遗忘旧信息 + 写入新信息
        c_t = f_gate * c_prev + i_gate * c_tilde
        
        # ========================================
        # Step 5: 计算隐状态输出
        # h_t = o_t * tanh(c_t)
        # ========================================
        h_t = o_gate * np.tanh(c_t)
        
        # 保存中间值（用于可视化和反向传播）
        self.cache = {
            'concat': concat,
            'f_gate': f_gate,
            'i_gate': i_gate,
            'c_tilde': c_tilde,
            'o_gate': o_gate,
            'c_prev': c_prev,
            'c_t': c_t,
            'h_t': h_t
        }
        
        return h_t, c_t


# 测试 LSTM Cell
print("=" * 60)
print("LSTM Cell 测试")
print("=" * 60)

input_size = 4
hidden_size = 8

lstm_cell = LSTMCell(input_size, hidden_size)

# 初始状态
h_0 = np.zeros(hidden_size)
c_0 = np.zeros(hidden_size)

# 输入
x = np.random.randn(input_size)

# 前向传播
h_1, c_1 = lstm_cell.forward(x, h_0, c_0)

print(f"\n输入维度: {input_size}")
print(f"隐藏维度: {hidden_size}")
print(f"\n输入 x: {x[:3]}... (前3个元素)")
print(f"初始 h_0: 全零")
print(f"初始 c_0: 全零")
print(f"\n输出 h_1: {h_1[:3]}... (前3个元素)")
print(f"输出 c_1: {c_1[:3]}... (前3个元素)")
print(f"\n各门的平均值:")
print(f"  遗忘门 f: {lstm_cell.cache['f_gate'].mean():.4f}")
print(f"  输入门 i: {lstm_cell.cache['i_gate'].mean():.4f}")
print(f"  输出门 o: {lstm_cell.cache['o_gate'].mean():.4f}")

### 3.2 完整 LSTM 类

In [ ]:
class LSTM:
    """
    完整的 LSTM 网络（处理序列）
    
    参数:
        input_size: 输入特征维度
        hidden_size: 隐藏状态维度
    """
    
    def __init__(self, input_size, hidden_size):
        self.cell = LSTMCell(input_size, hidden_size)
        self.hidden_size = hidden_size
    
    def forward(self, X, h_0=None, c_0=None):
        """
        处理整个序列
        
        参数:
            X: 输入序列, shape: (seq_len, input_size)
            h_0: 初始隐状态, shape: (hidden_size,)
            c_0: 初始细胞状态, shape: (hidden_size,)
        
        返回:
            h_all: 所有时间步的隐状态, shape: (seq_len, hidden_size)
            (h_last, c_last): 最后的状态
        """
        seq_len = X.shape[0]
        
        # 初始化状态
        if h_0 is None:
            h_0 = np.zeros(self.hidden_size)
        if c_0 is None:
            c_0 = np.zeros(self.hidden_size)
        
        h_t = h_0
        c_t = c_0
        
        # 存储所有时间步的输出
        h_all = []
        
        # 逐时间步处理
        for t in range(seq_len):
            x_t = X[t]
            h_t, c_t = self.cell.forward(x_t, h_t, c_t)
            h_all.append(h_t)
        
        return np.array(h_all), (h_t, c_t)


# 测试完整 LSTM
print("=" * 60)
print("LSTM 序列处理测试")
print("=" * 60)

seq_len = 10
input_size = 4
hidden_size = 8

lstm = LSTM(input_size, hidden_size)

# 生成随机序列
X = np.random.randn(seq_len, input_size)

# 前向传播
h_all, (h_last, c_last) = lstm.forward(X)

print(f"\n输入序列形状: {X.shape}")
print(f"所有隐状态形状: {h_all.shape}")
print(f"最终隐状态形状: {h_last.shape}")
print(f"最终细胞状态形状: {c_last.shape}")

### 3.3 与 PyTorch 对比验证

In [ ]:
def compare_with_pytorch():
    """
    比较 NumPy LSTM 与 PyTorch LSTM 的输出
    """
    print("=" * 60)
    print("NumPy LSTM vs PyTorch LSTM 对比")
    print("=" * 60)
    
    seq_len = 5
    input_size = 3
    hidden_size = 4
    
    # 固定随机种子
    np.random.seed(42)
    torch.manual_seed(42)
    
    # 创建 PyTorch LSTM
    lstm_pt = nn.LSTMCell(input_size, hidden_size)
    
    # 创建 NumPy LSTM 并复制权重
    lstm_np = LSTMCell(input_size, hidden_size)
    
    # 复制 PyTorch 的权重到 NumPy
    # PyTorch LSTMCell 的权重: weight_ih (4H, I), weight_hh (4H, H)
    # 需要拼接成 (4H, H+I)
    W_ih = lstm_pt.weight_ih.detach().numpy()  # (4H, I)
    W_hh = lstm_pt.weight_hh.detach().numpy()  # (4H, H)
    lstm_np.W = np.concatenate([W_hh, W_ih], axis=1)  # (4H, H+I)
    lstm_np.b = (lstm_pt.bias_ih.detach().numpy() + 
                 lstm_pt.bias_hh.detach().numpy())  # (4H,)
    
    # 生成输入
    X_np = np.random.randn(seq_len, input_size).astype(np.float32)
    X_pt = torch.from_numpy(X_np)
    
    # NumPy 前向传播
    h_np = np.zeros(hidden_size, dtype=np.float32)
    c_np = np.zeros(hidden_size, dtype=np.float32)
    h_np_all = []
    
    for t in range(seq_len):
        h_np, c_np = lstm_np.forward(X_np[t], h_np, c_np)
        h_np_all.append(h_np.copy())
    
    # PyTorch 前向传播
    h_pt = torch.zeros(hidden_size)
    c_pt = torch.zeros(hidden_size)
    h_pt_all = []
    
    for t in range(seq_len):
        h_pt, c_pt = lstm_pt(X_pt[t:t+1], (h_pt.unsqueeze(0), c_pt.unsqueeze(0)))
        h_pt = h_pt.squeeze(0)
        c_pt = c_pt.squeeze(0)
        h_pt_all.append(h_pt.detach().numpy().copy())
    
    # 比较结果
    h_np_all = np.array(h_np_all)
    h_pt_all = np.array(h_pt_all)
    
    diff = np.abs(h_np_all - h_pt_all)
    
    print(f"\n序列长度: {seq_len}")
    print(f"输入维度: {input_size}")
    print(f"隐藏维度: {hidden_size}")
    print(f"\n输出差异统计:")
    print(f"  最大差异: {diff.max():.2e}")
    print(f"  平均差异: {diff.mean():.2e}")
    
    if diff.max() < 1e-5:
        print("\n✓ NumPy 实现与 PyTorch 一致！")
    else:
        print("\n✗ 存在差异，请检查实现")
    
    # 可视化
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # 左图：隐状态对比
    ax = axes[0]
    for i in range(min(4, hidden_size)):
        ax.plot(h_np_all[:, i], 'o-', label=f'NumPy h[{i}]', alpha=0.7)
        ax.plot(h_pt_all[:, i], 's--', label=f'PyTorch h[{i}]', alpha=0.7)
    ax.set_xlabel('时间步', fontsize=12)
    ax.set_ylabel('隐状态值', fontsize=12)
    ax.set_title('隐状态对比（前4个维度）', fontsize=12)
    ax.legend(loc='best', fontsize=8, ncol=2)
    ax.grid(True, alpha=0.3)
    
    # 右图：差异
    ax = axes[1]
    im = ax.imshow(diff.T, aspect='auto', cmap='Reds')
    ax.set_xlabel('时间步', fontsize=12)
    ax.set_ylabel('隐状态维度', fontsize=12)
    ax.set_title('NumPy vs PyTorch 差异热图', fontsize=12)
    plt.colorbar(im, ax=ax, label='绝对差异')
    
    plt.tight_layout()
    plt.show()


compare_with_pytorch()

---

## 第四部分：LSTM 如何解决梯度消失

### 4.1 细胞状态的梯度分析

In [ ]:
def analyze_lstm_gradient():
    """
    分析 LSTM 的梯度流动
    """
    print("=" * 60)
    print("LSTM 梯度分析：为什么不会消失？")
    print("=" * 60)
    
    # 关键公式分析
    analysis = """
    
┌─────────────────────────────────────────────────────────────┐
│                    LSTM 细胞状态更新                        │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│    c_t = f_t * c_{t-1} + i_t * c̃_t                          │
│                                                             │
│    梯度:                                                    │
│    ∂c_t / ∂c_{t-1} = f_t                                    │
│                                                             │
│    沿时间传播:                                              │
│    ∂c_T / ∂c_k = ∏(t=k+1 to T) f_t                         │
│                                                             │
└─────────────────────────────────────────────────────────────┘

关键洞察:

1. RNN 的问题: ∂h_t/∂h_{t-1} = W_hh * diag(1-h²)
   - 必须乘以权重矩阵 W_hh
   - 还要乘以 tanh 导数 (1-h²)
   - 连乘后容易消失/爆炸

2. LSTM 的解决: ∂c_t/∂c_{t-1} = f_t
   - 只乘以遗忘门 f_t
   - f_t ∈ [0, 1]，由网络学习
   - 如果 f_t ≈ 1，梯度几乎无损传递！

3. 为什么有效:
   - 细胞状态更新是"加法"而非"乘法"
   - 梯度可以通过"高速公路"直接传递
   - 网络可以学习何时保持梯度（f_t→1）
    """
    
    print(analysis)
    
    # 可视化对比
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    seq_len = 100
    
    # 左图：RNN 梯度衰减
    ax = axes[0]
    
    # RNN: 假设 ||W_hh * diag(1-h²)|| ≈ 0.9
    rnn_factor = 0.9
    rnn_gradient = rnn_factor ** np.arange(seq_len)
    
    ax.semilogy(range(seq_len), rnn_gradient, 'r-', linewidth=2, label='RNN 梯度')
    ax.axhline(y=1e-10, color='gray', linestyle='--', alpha=0.5)
    ax.fill_between(range(seq_len), 1e-15, rnn_gradient, alpha=0.2, color='red')
    
    ax.set_xlabel('时间步差', fontsize=12)
    ax.set_ylabel('梯度大小', fontsize=12)
    ax.set_title('RNN: 梯度指数衰减', fontsize=14, color='red')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim(1e-15, 10)
    
    # 右图：LSTM 梯度保持
    ax = axes[1]
    
    # LSTM: 不同的遗忘门值
    forget_gates = [0.99, 0.95, 0.9, 0.8]
    colors = ['green', 'blue', 'orange', 'red']
    
    for fg, color in zip(forget_gates, colors):
        lstm_gradient = fg ** np.arange(seq_len)
        ax.semilogy(range(seq_len), lstm_gradient, '-', color=color, 
                    linewidth=2, label=f'f_t = {fg}')
    
    ax.axhline(y=1e-10, color='gray', linestyle='--', alpha=0.5)
    
    ax.set_xlabel('时间步差', fontsize=12)
    ax.set_ylabel('梯度大小', fontsize=12)
    ax.set_title('LSTM: 遗忘门控制梯度流动', fontsize=14, color='green')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    ax.set_ylim(1e-15, 10)
    
    # 添加说明
    ax.text(50, 1, '当 f_t → 1 时\n梯度几乎无损传递', 
            ha='center', fontsize=11, color='green',
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
    
    plt.tight_layout()
    plt.show()


analyze_lstm_gradient()

### 4.2 长期依赖任务对比：RNN vs LSTM

In [ ]:
def compare_rnn_lstm_memory():
    """
    在长期依赖任务上对比 RNN 和 LSTM
    
    任务：记住序列开头的符号（+1 或 -1）
    """
    print("=" * 60)
    print("长期依赖任务：RNN vs LSTM")
    print("=" * 60)
    
    def generate_data(n_samples, seq_len):
        """生成数据：第一个位置是标签，其余为噪声"""
        X = np.random.randn(n_samples, seq_len, 1) * 0.1  # 小噪声
        Y = np.zeros((n_samples, 1))
        
        for i in range(n_samples):
            label = np.random.choice([-1, 1])
            X[i, 0, 0] = label  # 第一个位置是信号
            Y[i, 0] = label
        
        return torch.FloatTensor(X), torch.FloatTensor(Y)
    
    # 模型定义
    class MemoryModel(nn.Module):
        def __init__(self, model_type, hidden_size):
            super().__init__()
            if model_type == 'RNN':
                self.rnn = nn.RNN(1, hidden_size, batch_first=True)
            else:
                self.rnn = nn.LSTM(1, hidden_size, batch_first=True)
            self.fc = nn.Linear(hidden_size, 1)
            self.model_type = model_type
        
        def forward(self, x):
            out, _ = self.rnn(x)
            return torch.tanh(self.fc(out[:, -1, :]))
    
    # 测试不同序列长度
    seq_lengths = [10, 25, 50, 100]
    hidden_size = 32
    n_train = 500
    n_test = 100
    n_epochs = 100
    
    results = {'RNN': {}, 'LSTM': {}}
    
    for model_type in ['RNN', 'LSTM']:
        print(f"\n训练 {model_type}...")
        
        for seq_len in seq_lengths:
            # 生成数据
            X_train, Y_train = generate_data(n_train, seq_len)
            X_test, Y_test = generate_data(n_test, seq_len)
            
            # 创建模型
            torch.manual_seed(42)
            model = MemoryModel(model_type, hidden_size)
            optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
            criterion = nn.MSELoss()
            
            # 训练
            for epoch in range(n_epochs):
                model.train()
                optimizer.zero_grad()
                output = model(X_train)
                loss = criterion(output, Y_train)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
            
            # 测试
            model.eval()
            with torch.no_grad():
                pred = model(X_test)
                acc = ((pred > 0) == (Y_test > 0)).float().mean().item()
            
            results[model_type][seq_len] = acc
            print(f"  seq_len={seq_len:3d}: 准确率 = {acc*100:.1f}%")
    
    # 可视化
    fig, ax = plt.subplots(figsize=(10, 6))
    
    x = np.arange(len(seq_lengths))
    width = 0.35
    
    rnn_accs = [results['RNN'][s] * 100 for s in seq_lengths]
    lstm_accs = [results['LSTM'][s] * 100 for s in seq_lengths]
    
    bars1 = ax.bar(x - width/2, rnn_accs, width, label='RNN', color='coral', edgecolor='black')
    bars2 = ax.bar(x + width/2, lstm_accs, width, label='LSTM', color='steelblue', edgecolor='black')
    
    ax.axhline(y=50, color='gray', linestyle='--', label='随机猜测', alpha=0.7)
    ax.axhline(y=90, color='green', linestyle='--', label='良好性能', alpha=0.5)
    
    ax.set_xlabel('序列长度', fontsize=12)
    ax.set_ylabel('准确率 (%)', fontsize=12)
    ax.set_title('长期记忆任务：RNN vs LSTM', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels([str(s) for s in seq_lengths])
    ax.legend()
    ax.set_ylim(0, 105)
    ax.grid(True, alpha=0.3, axis='y')
    
    # 添加数值标签
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2, height + 1, 
                    f'{height:.0f}%', ha='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    # 结论
    print("\n" + "=" * 60)
    print("结论:")
    print("-" * 60)
    print("• RNN 在长序列上性能急剧下降（接近随机猜测）")
    print("• LSTM 能保持较高准确率，成功记住开头信息")
    print("• 这验证了 LSTM 的门控机制能有效解决梯度消失")
    
    return results


memory_results = compare_rnn_lstm_memory()

---

## 第五部分：可视化门的行为

### 5.1 观察实际序列中的门激活

In [ ]:
def visualize_gate_activations():
    """
    可视化 LSTM 各门在处理序列时的激活模式
    """
    print("=" * 60)
    print("LSTM 门激活可视化")
    print("=" * 60)
    
    # 创建带有钩子的 LSTM 来捕获门激活
    class LSTMWithGateAccess(nn.Module):
        def __init__(self, input_size, hidden_size):
            super().__init__()
            self.lstm = nn.LSTMCell(input_size, hidden_size)
            self.hidden_size = hidden_size
            
            # 存储门激活
            self.gate_history = {'f': [], 'i': [], 'o': []}
        
        def forward(self, x):
            """手动前向以捕获门"""
            seq_len = x.size(1)
            batch_size = x.size(0)
            H = self.hidden_size
            
            h = torch.zeros(batch_size, H)
            c = torch.zeros(batch_size, H)
            
            self.gate_history = {'f': [], 'i': [], 'o': []}
            
            for t in range(seq_len):
                x_t = x[:, t, :]
                
                # 计算门（手动实现以获取中间值）
                gates = (x_t @ self.lstm.weight_ih.T + 
                        h @ self.lstm.weight_hh.T + 
                        self.lstm.bias_ih + self.lstm.bias_hh)
                
                i_gate = torch.sigmoid(gates[:, 0*H:1*H])
                f_gate = torch.sigmoid(gates[:, 1*H:2*H])
                g = torch.tanh(gates[:, 2*H:3*H])
                o_gate = torch.sigmoid(gates[:, 3*H:4*H])
                
                c = f_gate * c + i_gate * g
                h = o_gate * torch.tanh(c)
                
                # 记录门激活（取平均）
                self.gate_history['f'].append(f_gate.mean().item())
                self.gate_history['i'].append(i_gate.mean().item())
                self.gate_history['o'].append(o_gate.mean().item())
            
            return h
    
    # 创建一个有模式的序列
    seq_len = 50
    input_size = 1
    hidden_size = 16
    
    # 输入：正弦波 + 脉冲信号
    t = np.linspace(0, 4*np.pi, seq_len)
    signal = np.sin(t) * 0.5
    signal[10] = 2.0   # 在 t=10 处有个重要脉冲
    signal[30] = -2.0  # 在 t=30 处有个负脉冲
    
    X = torch.FloatTensor(signal).view(1, seq_len, 1)
    
    # 创建并运行模型
    torch.manual_seed(42)
    model = LSTMWithGateAccess(input_size, hidden_size)
    
    with torch.no_grad():
        _ = model(X)
    
    # 可视化
    fig, axes = plt.subplots(4, 1, figsize=(12, 10), sharex=True)
    
    # 输入信号
    ax = axes[0]
    ax.plot(range(seq_len), signal, 'b-', linewidth=2)
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.scatter([10, 30], [signal[10], signal[30]], color='red', s=100, zorder=5)
    ax.set_ylabel('输入', fontsize=11)
    ax.set_title('输入序列（红点为重要事件）', fontsize=12)
    ax.grid(True, alpha=0.3)
    
    # 遗忘门
    ax = axes[1]
    ax.plot(range(seq_len), model.gate_history['f'], 'r-', linewidth=2)
    ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
    ax.set_ylabel('遗忘门', fontsize=11)
    ax.set_title('遗忘门 f_t: 高=保留旧信息，低=遗忘', fontsize=12)
    ax.set_ylim(0, 1)
    ax.grid(True, alpha=0.3)
    
    # 输入门
    ax = axes[2]
    ax.plot(range(seq_len), model.gate_history['i'], 'g-', linewidth=2)
    ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
    ax.set_ylabel('输入门', fontsize=11)
    ax.set_title('输入门 i_t: 高=写入新信息，低=忽略', fontsize=12)
    ax.set_ylim(0, 1)
    ax.grid(True, alpha=0.3)
    
    # 输出门
    ax = axes[3]
    ax.plot(range(seq_len), model.gate_history['o'], 'purple', linewidth=2)
    ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
    ax.set_ylabel('输出门', fontsize=11)
    ax.set_xlabel('时间步', fontsize=11)
    ax.set_title('输出门 o_t: 高=输出记忆，低=隐藏', fontsize=12)
    ax.set_ylim(0, 1)
    ax.grid(True, alpha=0.3)
    
    # 标注重要事件
    for ax in axes[1:]:
        ax.axvline(x=10, color='red', linestyle=':', alpha=0.5)
        ax.axvline(x=30, color='red', linestyle=':', alpha=0.5)
    
    plt.tight_layout()
    plt.show()
    
    print("\n观察:")
    print("-" * 40)
    print("• 门的激活值随输入动态变化")
    print("• 在重要事件（脉冲）附近，门的响应可能不同")
    print("• 这种自适应行为是 LSTM 的核心优势")


visualize_gate_activations()

---

## 本章总结

### 核心要点

1. **LSTM 的两大创新**：
   - 细胞状态 $c_t$：信息高速公路，梯度可以直接传递
   - 门控机制：学习何时记住/遗忘/输出

2. **三个门的作用**：
   - 遗忘门 $f_t$：决定丢弃哪些旧信息
   - 输入门 $i_t$：决定写入哪些新信息
   - 输出门 $o_t$：决定输出哪些信息

3. **解决梯度消失的关键**：
   - 细胞状态更新是**加法**：$c_t = f_t \odot c_{t-1} + i_t \odot \tilde{c}_t$
   - 梯度沿细胞状态传递：$\frac{\partial c_t}{\partial c_{t-1}} = f_t$
   - 当 $f_t \to 1$ 时，梯度无损传递

### 代码模板

```python
# PyTorch LSTM 使用
lstm = nn.LSTM(input_size, hidden_size, num_layers=1, batch_first=True)
output, (h_n, c_n) = lstm(input_seq)

# 或使用 LSTMCell 逐步处理
lstm_cell = nn.LSTMCell(input_size, hidden_size)
h, c = lstm_cell(x_t, (h_prev, c_prev))
```

---

## 下一章预告

**第 6 章：GRU 简化设计**

GRU（Gated Recurrent Unit）是 LSTM 的简化版本：
- 只有两个门（重置门和更新门）
- 没有单独的细胞状态
- 参数更少，训练更快
- 在很多任务上效果与 LSTM 相当

👉 [06_gru_simplified.ipynb](./06_gru_simplified.ipynb)